In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import precision_score, recall_score, roc_auc_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier

# === Load synthetic data (label already binarized) ===
df_synth = pd.read_csv("generated_data_Our_prompts_MIMIC.csv")
if 'label' not in df_synth.columns:
    df_synth["label"] = df_synth["los_seconds"].astype(int)

# === Split synthetic data only (80% train / 20% test) ===
df_train, df_test = train_test_split(
    df_synth, test_size=0.2, stratify=df_synth["label"], random_state=42
)

# === Preserve unencoded race column
df_train["race_original"] = df_train["race"]
df_test["race_original"] = df_test["race"]

# === Separate features and label
X_train = df_train.drop(columns=["label", "los_seconds"])
y_train = df_train["label"]
X_test = df_test.drop(columns=["label", "los_seconds"])
y_test = df_test["label"]

# === Drop identifier columns not useful for training
drop_cols = ["stay_id", "subject_id", "hadm_id"]
X_train = X_train.drop(columns=[col for col in drop_cols if col in X_train.columns])
X_test = X_test.drop(columns=[col for col in drop_cols if col in X_test.columns])

# === Encode non-numeric columns
for X in [X_train, X_test]:
    non_numeric_cols = X.select_dtypes(include=["object", "bool"]).columns
    for col in non_numeric_cols:
        if set(X[col].dropna().unique()).issubset({'True', 'False', 'true', 'false'}):
            X[col] = X[col].astype(str).map({'True': 1, 'False': 0, 'true': 1, 'false': 0})
        else:
            X[col] = LabelEncoder().fit_transform(X[col].astype(str))

# === Impute missing values
X_train = pd.DataFrame(SimpleImputer(strategy='most_frequent').fit_transform(X_train), columns=X_train.columns)
X_test = pd.DataFrame(SimpleImputer(strategy='most_frequent').fit_transform(X_test), columns=X_test.columns)

# === Downcast for memory efficiency
X_train = X_train.apply(pd.to_numeric, downcast='float')
X_test = X_test.apply(pd.to_numeric, downcast='float')

# === Sensitive attribute
sensitive_col = "race_original"
privileged_value = "WHITE"

# === Define models
models = {
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "SVM": SVC(probability=True, random_state=42),
    "XGBoost": XGBClassifier(eval_metric="logloss", random_state=42)
}

# === Evaluation setup
N_REPEATS = 5
results = {model: {"Precision": [], "Recall": [], "AUC": [], "FTU": [], "DP": []} for model in models}

# === Evaluation loop
for seed in range(N_REPEATS):
    X_train_shuffled, y_train_shuffled = shuffle(X_train, y_train, random_state=seed)
    race_test = df_test[sensitive_col]

    for name, model in models.items():
        model.fit(X_train_shuffled, y_train_shuffled)
        y_pred = model.predict(X_test)
        y_prob = model.predict_proba(X_test)[:, 1] if hasattr(model, "predict_proba") else None

        results[name]["Precision"].append(precision_score(y_test, y_pred))
        results[name]["Recall"].append(recall_score(y_test, y_pred))
        results[name]["AUC"].append(roc_auc_score(y_test, y_prob) if y_prob is not None else np.nan)

        # FTU (flip WHITE <-> BLACK)
        flipped_race = race_test.replace({
            "WHITE": "BLACK/AFRICAN AMERICAN",
            "BLACK/AFRICAN AMERICAN": "WHITE"
        })
        X_test_flipped = X_test.copy()
        if "race" in X_test_flipped.columns:
            X_test_flipped["race"] = LabelEncoder().fit_transform(flipped_race.astype(str))
        y_pred_flipped = model.predict(X_test_flipped)
        ftu = np.mean(np.abs(y_pred - y_pred_flipped))
        results[name]["FTU"].append(ftu)

        # DP (Demographic Parity)
        mask_priv = race_test == privileged_value
        mask_unpriv = ~mask_priv
        p_priv = y_pred[mask_priv].mean() if np.any(mask_priv) else 0
        p_unpriv = y_pred[mask_unpriv].mean() if np.any(mask_unpriv) else 0
        dp = abs(p_priv - p_unpriv)
        results[name]["DP"].append(dp)

# === Format and print results
def format_metric(values):
    return f"{np.mean(values):.3f} ± {np.std(values):.3f}"

print(f"{'Model':<20} {'Precision↑':<15} {'Recall↑':<15} {'AUROC↑':<15} {'FTU↓':<15} {'DP↓':<15}")
print("-" * 95)
table_rows = []
for model in models:
    row = {"Model": model}
    line = [model]
    for metric in ["Precision", "Recall", "AUC", "FTU", "DP"]:
        formatted = format_metric(results[model][metric])
        row[metric] = formatted
        line.append(formatted)
    print(f"{line[0]:<20} {line[1]:<15} {line[2]:<15} {line[3]:<15} {line[4]:<15} {line[5]:<15}")
    table_rows.append(row)

# === Save to CSV
results_df = pd.DataFrame(table_rows)
results_df.to_csv("fairness_results_our.csv", index=False)
print("\n✅ Saved: fairness_results_our.csv")


Model                Precision↑      Recall↑         AUROC↑          FTU↓            DP↓            
-----------------------------------------------------------------------------------------------
Decision Tree        0.900 ± 0.000   0.872 ± 0.000   0.537 ± 0.000   0.020 ± 0.000   0.039 ± 0.000  
Logistic Regression  0.898 ± 0.000   0.995 ± 0.000   0.702 ± 0.000   0.003 ± 0.000   0.006 ± 0.000  
Random Forest        0.898 ± 0.001   0.985 ± 0.002   0.689 ± 0.008   0.008 ± 0.003   0.005 ± 0.005  
SVM                  0.892 ± 0.000   1.000 ± 0.000   0.608 ± 0.006   0.000 ± 0.000   0.000 ± 0.000  
XGBoost              0.905 ± 0.000   0.969 ± 0.000   0.644 ± 0.000   0.025 ± 0.000   0.001 ± 0.000  

✅ Saved: fairness_results_our.csv


In [16]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import precision_score, recall_score, roc_auc_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier

# === Load synthetic data (label already binarized) ===
df_synth = pd.read_csv("generated_data_CLLM_prompt_Mimic.csv")
if 'label' not in df_synth.columns:
    df_synth["label"] = df_synth["los_seconds"].astype(int)

# === Split synthetic data only (80% train / 20% test) ===
df_train, df_test = train_test_split(
    df_synth, test_size=0.2, stratify=df_synth["label"], random_state=42
)

# === Preserve unencoded race column
df_train["race_original"] = df_train["race"]
df_test["race_original"] = df_test["race"]

# === Separate features and label
X_train = df_train.drop(columns=["label", "los_seconds"])
y_train = df_train["label"]
X_test = df_test.drop(columns=["label", "los_seconds"])
y_test = df_test["label"]

# === Drop identifier columns not useful for training
drop_cols = ["stay_id", "subject_id", "hadm_id"]
X_train = X_train.drop(columns=[col for col in drop_cols if col in X_train.columns])
X_test = X_test.drop(columns=[col for col in drop_cols if col in X_test.columns])

# === Encode non-numeric columns
for X in [X_train, X_test]:
    non_numeric_cols = X.select_dtypes(include=["object", "bool"]).columns
    for col in non_numeric_cols:
        if set(X[col].dropna().unique()).issubset({'True', 'False', 'true', 'false'}):
            X[col] = X[col].astype(str).map({'True': 1, 'False': 0, 'true': 1, 'false': 0})
        else:
            X[col] = LabelEncoder().fit_transform(X[col].astype(str))

# === Impute missing values
X_train = pd.DataFrame(SimpleImputer(strategy='most_frequent').fit_transform(X_train), columns=X_train.columns)
X_test = pd.DataFrame(SimpleImputer(strategy='most_frequent').fit_transform(X_test), columns=X_test.columns)

# === Downcast for memory efficiency
X_train = X_train.apply(pd.to_numeric, downcast='float')
X_test = X_test.apply(pd.to_numeric, downcast='float')

# === Sensitive attribute
sensitive_col = "race_original"
privileged_value = "WHITE"

# === Define models
models = {
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "SVM": SVC(probability=True, random_state=42),
    "XGBoost": XGBClassifier(eval_metric="logloss", random_state=42)
}

# === Evaluation setup
N_REPEATS = 5
results = {model: {"Precision": [], "Recall": [], "AUC": [], "FTU": [], "DP": []} for model in models}

# === Evaluation loop
for seed in range(N_REPEATS):
    X_train_shuffled, y_train_shuffled = shuffle(X_train, y_train, random_state=seed)
    race_test = df_test[sensitive_col]

    for name, model in models.items():
        model.fit(X_train_shuffled, y_train_shuffled)
        y_pred = model.predict(X_test)
        y_prob = model.predict_proba(X_test)[:, 1] if hasattr(model, "predict_proba") else None

        results[name]["Precision"].append(precision_score(y_test, y_pred))
        results[name]["Recall"].append(recall_score(y_test, y_pred))
        results[name]["AUC"].append(roc_auc_score(y_test, y_prob) if y_prob is not None else np.nan)

        # FTU (flip WHITE <-> BLACK)
        flipped_race = race_test.replace({
            "WHITE": "BLACK/AFRICAN AMERICAN",
            "BLACK/AFRICAN AMERICAN": "WHITE"
        })
        X_test_flipped = X_test.copy()
        if "race" in X_test_flipped.columns:
            X_test_flipped["race"] = LabelEncoder().fit_transform(flipped_race.astype(str))
        y_pred_flipped = model.predict(X_test_flipped)
        ftu = np.mean(np.abs(y_pred - y_pred_flipped))
        results[name]["FTU"].append(ftu)

        # DP (Demographic Parity)
        mask_priv = race_test == privileged_value
        mask_unpriv = ~mask_priv
        p_priv = y_pred[mask_priv].mean() if np.any(mask_priv) else 0
        p_unpriv = y_pred[mask_unpriv].mean() if np.any(mask_unpriv) else 0
        dp = abs(p_priv - p_unpriv)
        results[name]["DP"].append(dp)

# === Format and print results
def format_metric(values):
    return f"{np.mean(values):.3f} ± {np.std(values):.3f}"

print(f"{'Model':<20} {'Precision↑':<15} {'Recall↑':<15} {'AUROC↑':<15} {'FTU↓':<15} {'DP↓':<15}")
print("-" * 95)
table_rows = []
for model in models:
    row = {"Model": model}
    line = [model]
    for metric in ["Precision", "Recall", "AUC", "FTU", "DP"]:
        formatted = format_metric(results[model][metric])
        row[metric] = formatted
        line.append(formatted)
    print(f"{line[0]:<20} {line[1]:<15} {line[2]:<15} {line[3]:<15} {line[4]:<15} {line[5]:<15}")
    table_rows.append(row)

# === Save to CSV
results_df = pd.DataFrame(table_rows)
results_df.to_csv("fairness_results_mimic.csv", index=False)
print("\n✅ Saved: fairness_results_mimic.csv")


Model                Precision↑      Recall↑         AUROC↑          FTU↓            DP↓            
-----------------------------------------------------------------------------------------------
Decision Tree        0.660 ± 0.000   0.604 ± 0.000   0.558 ± 0.000   0.029 ± 0.000   0.044 ± 0.000  
Logistic Regression  0.682 ± 0.000   0.863 ± 0.000   0.674 ± 0.000   0.003 ± 0.000   0.028 ± 0.000  
Random Forest        0.669 ± 0.004   0.809 ± 0.002   0.632 ± 0.005   0.073 ± 0.016   0.014 ± 0.007  
SVM                  0.641 ± 0.000   0.930 ± 0.000   0.671 ± 0.000   0.000 ± 0.000   0.010 ± 0.000  
XGBoost              0.684 ± 0.000   0.736 ± 0.000   0.639 ± 0.000   0.081 ± 0.000   0.024 ± 0.000  

✅ Saved: fairness_results_mimic.csv


In [15]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import precision_score, recall_score, roc_auc_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier

# === Load synthetic data (label already binarized) ===
df_synth = pd.read_csv("mimic_synthetic_data_3400_samples_DECAF.csv")
if 'label' not in df_synth.columns:
    df_synth["label"] = (df_synth["los_seconds"] >= 345600).astype(int)

# === Split synthetic data only (80% train / 20% test) ===
df_train, df_test = train_test_split(
    df_synth, test_size=0.2, stratify=df_synth["label"], random_state=42
)

# === Preserve unencoded race column
df_train["race_original"] = df_train["race"]
df_test["race_original"] = df_test["race"]

# === Separate features and label
X_train = df_train.drop(columns=["label", "los_seconds"])
y_train = df_train["label"]
X_test = df_test.drop(columns=["label", "los_seconds"])
y_test = df_test["label"]

# === Drop identifier columns not useful for training
drop_cols = ["stay_id", "subject_id", "hadm_id"]
X_train = X_train.drop(columns=[col for col in drop_cols if col in X_train.columns])
X_test = X_test.drop(columns=[col for col in drop_cols if col in X_test.columns])

# === Encode non-numeric columns
for X in [X_train, X_test]:
    non_numeric_cols = X.select_dtypes(include=["object", "bool"]).columns
    for col in non_numeric_cols:
        if set(X[col].dropna().unique()).issubset({'True', 'False', 'true', 'false'}):
            X[col] = X[col].astype(str).map({'True': 1, 'False': 0, 'true': 1, 'false': 0})
        else:
            X[col] = LabelEncoder().fit_transform(X[col].astype(str))

# === Impute missing values
X_train = pd.DataFrame(SimpleImputer(strategy='most_frequent').fit_transform(X_train), columns=X_train.columns)
X_test = pd.DataFrame(SimpleImputer(strategy='most_frequent').fit_transform(X_test), columns=X_test.columns)

# === Downcast for memory efficiency
X_train = X_train.apply(pd.to_numeric, downcast='float')
X_test = X_test.apply(pd.to_numeric, downcast='float')

# === Sensitive attribute
sensitive_col = "race_original"
privileged_value = "WHITE"

# === Define models
models = {
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "SVM": SVC(probability=True, random_state=42),
    "XGBoost": XGBClassifier(eval_metric="logloss", random_state=42)
}

# === Evaluation setup
N_REPEATS = 5
results = {model: {"Precision": [], "Recall": [], "AUC": [], "FTU": [], "DP": []} for model in models}

# === Evaluation loop
for seed in range(N_REPEATS):
    X_train_shuffled, y_train_shuffled = shuffle(X_train, y_train, random_state=seed)
    race_test = df_test[sensitive_col]

    for name, model in models.items():
        model.fit(X_train_shuffled, y_train_shuffled)
        y_pred = model.predict(X_test)
        y_prob = model.predict_proba(X_test)[:, 1] if hasattr(model, "predict_proba") else None

        results[name]["Precision"].append(precision_score(y_test, y_pred))
        results[name]["Recall"].append(recall_score(y_test, y_pred))
        results[name]["AUC"].append(roc_auc_score(y_test, y_prob) if y_prob is not None else np.nan)

        # FTU (flip WHITE <-> BLACK)
        flipped_race = race_test.replace({
            "WHITE": "BLACK/AFRICAN AMERICAN",
            "BLACK/AFRICAN AMERICAN": "WHITE"
        })
        X_test_flipped = X_test.copy()
        if "race" in X_test_flipped.columns:
            X_test_flipped["race"] = LabelEncoder().fit_transform(flipped_race.astype(str))
        y_pred_flipped = model.predict(X_test_flipped)
        ftu = np.mean(np.abs(y_pred - y_pred_flipped))
        results[name]["FTU"].append(ftu)

        # DP (Demographic Parity)
        mask_priv = race_test == privileged_value
        mask_unpriv = ~mask_priv
        p_priv = y_pred[mask_priv].mean() if np.any(mask_priv) else 0
        p_unpriv = y_pred[mask_unpriv].mean() if np.any(mask_unpriv) else 0
        dp = abs(p_priv - p_unpriv)
        results[name]["DP"].append(dp)

# === Format and print results
def format_metric(values):
    return f"{np.mean(values):.3f} ± {np.std(values):.3f}"

print(f"{'Model':<20} {'Precision↑':<15} {'Recall↑':<15} {'AUROC↑':<15} {'FTU↓':<15} {'DP↓':<15}")
print("-" * 95)
table_rows = []
for model in models:
    row = {"Model": model}
    line = [model]
    for metric in ["Precision", "Recall", "AUC", "FTU", "DP"]:
        formatted = format_metric(results[model][metric])
        row[metric] = formatted
        line.append(formatted)
    print(f"{line[0]:<20} {line[1]:<15} {line[2]:<15} {line[3]:<15} {line[4]:<15} {line[5]:<15}")
    table_rows.append(row)

# === Save to CSV
results_df = pd.DataFrame(table_rows)
results_df.to_csv("fairness_results_decaf_mimic.csv", index=False)
print("\n✅ Saved: fairness_results_decaf_mimic.csv")


Model                Precision↑      Recall↑         AUROC↑          FTU↓            DP↓            
-----------------------------------------------------------------------------------------------
Decision Tree        0.593 ± 0.000   0.576 ± 0.000   0.673 ± 0.000   0.006 ± 0.000   0.021 ± 0.000  
Logistic Regression  0.714 ± 0.002   0.532 ± 0.000   0.820 ± 0.000   0.028 ± 0.001   0.034 ± 0.001  
Random Forest        0.681 ± 0.008   0.598 ± 0.018   0.831 ± 0.001   0.054 ± 0.004   0.038 ± 0.024  
SVM                  0.667 ± 0.000   0.376 ± 0.000   0.769 ± 0.000   0.003 ± 0.000   0.067 ± 0.000  
XGBoost              0.670 ± 0.000   0.600 ± 0.000   0.816 ± 0.000   0.097 ± 0.000   0.012 ± 0.000  

✅ Saved: fairness_results_decaf_mimic.csv


In [14]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import precision_score, recall_score, roc_auc_score
from xgboost import XGBClassifier

# === Load synthetic data (label already binarized) ===
df_synth = pd.read_csv("Real_MIMIC.csv")
if 'label' not in df_synth.columns:
    df_synth["label"] = (df_synth["los_seconds"] >= 345600).astype(int)

# === Split synthetic data only (80% train / 20% test) ===
df_train, df_test = train_test_split(
    df_synth, test_size=0.2, stratify=df_synth["label"], random_state=42
)

# === Preserve unencoded race column
df_train["race_original"] = df_train["race"]
df_test["race_original"] = df_test["race"]

# === Separate features and label
X_train = df_train.drop(columns=["label", "los_seconds"])
y_train = df_train["label"]
X_test = df_test.drop(columns=["label", "los_seconds"])
y_test = df_test["label"]

# === Drop identifier columns not useful for training
drop_cols = ["stay_id", "subject_id", "hadm_id"]
X_train = X_train.drop(columns=[col for col in drop_cols if col in X_train.columns])
X_test = X_test.drop(columns=[col for col in drop_cols if col in X_test.columns])

# === Encode non-numeric columns
for X in [X_train, X_test]:
    non_numeric_cols = X.select_dtypes(include=["object", "bool"]).columns
    for col in non_numeric_cols:
        if set(X[col].dropna().unique()).issubset({'True', 'False', 'true', 'false'}):
            X[col] = X[col].astype(str).map({'True': 1, 'False': 0, 'true': 1, 'false': 0})
        else:
            X[col] = LabelEncoder().fit_transform(X[col].astype(str))

# === Impute missing values
X_train = pd.DataFrame(SimpleImputer(strategy='most_frequent').fit_transform(X_train), columns=X_train.columns)
X_test = pd.DataFrame(SimpleImputer(strategy='most_frequent').fit_transform(X_test), columns=X_test.columns)

# === Downcast for memory efficiency
X_train = X_train.apply(pd.to_numeric, downcast='float')
X_test = X_test.apply(pd.to_numeric, downcast='float')

# === Sensitive attribute
sensitive_col = "race_original"
privileged_value = "WHITE"

# === Define only XGBoost model
model = XGBClassifier(eval_metric="logloss", random_state=42)

# === Evaluation setup
N_REPEATS = 5
results = {"XGBoost": {"Precision": [], "Recall": [], "AUC": [], "FTU": [], "DP": []}}

# === Evaluation loop
for seed in range(N_REPEATS):
    X_train_shuffled, y_train_shuffled = shuffle(X_train, y_train, random_state=seed)
    race_test = df_test[sensitive_col]

    model.fit(X_train_shuffled, y_train_shuffled)
    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1]

    results["XGBoost"]["Precision"].append(precision_score(y_test, y_pred))
    results["XGBoost"]["Recall"].append(recall_score(y_test, y_pred))
    results["XGBoost"]["AUC"].append(roc_auc_score(y_test, y_prob))

    # FTU (flip WHITE <-> BLACK)
    flipped_race = race_test.replace({
        "WHITE": "BLACK/AFRICAN AMERICAN",
        "BLACK/AFRICAN AMERICAN": "WHITE"
    })
    X_test_flipped = X_test.copy()
    if "race" in X_test_flipped.columns:
        X_test_flipped["race"] = LabelEncoder().fit_transform(flipped_race.astype(str))
    y_pred_flipped = model.predict(X_test_flipped)
    ftu = np.mean(np.abs(y_pred - y_pred_flipped))
    results["XGBoost"]["FTU"].append(ftu)

    # DP (Demographic Parity)
    mask_priv = race_test == privileged_value
    mask_unpriv = ~mask_priv
    p_priv = y_pred[mask_priv].mean() if np.any(mask_priv) else 0
    p_unpriv = y_pred[mask_unpriv].mean() if np.any(mask_unpriv) else 0
    dp = abs(p_priv - p_unpriv)
    results["XGBoost"]["DP"].append(dp)

# === Format and print results
def format_metric(values):
    return f"{np.mean(values):.3f} ± {np.std(values):.3f}"

print(f"{'Model':<20} {'Precision↑':<15} {'Recall↑':<15} {'AUROC↑':<15} {'FTU↓':<15} {'DP↓':<15}")
print("-" * 95)
row = ["XGBoost"]
for metric in ["Precision", "Recall", "AUC", "FTU", "DP"]:
    row.append(format_metric(results["XGBoost"][metric]))
print(f"{row[0]:<20} {row[1]:<15} {row[2]:<15} {row[3]:<15} {row[4]:<15} {row[5]:<15}")

# === Save to CSV
results_df = pd.DataFrame([{
    "Model": "XGBoost",
    "Precision": row[1],
    "Recall": row[2],
    "AUC": row[3],
    "FTU": row[4],
    "DP": row[5]
}])
results_df.to_csv("fairness_results_real_mimic.csv", index=False)
print("\n✅ Saved: fairness_results_real_mimic.csv")


Model                Precision↑      Recall↑         AUROC↑          FTU↓            DP↓            
-----------------------------------------------------------------------------------------------
XGBoost              0.703 ± 0.000   0.653 ± 0.000   0.853 ± 0.000   0.037 ± 0.000   0.074 ± 0.000  

✅ Saved: fairness_results_real_mimic.csv
